In [125]:
import sys
sys.path.append('/Users/toshan/Dropbox/GWR/PyGWRJing/PyGWR/')
from M_FBGWR_May2016 import FBGWR as FBGWR_Wei
sys.path.append('/Users/toshan/Dropbox/GWR/PyGWRJing/PyGWR/')
sys.path.append('/Users/toshan/dev/pysal/pysal/contrib/gwr/')
from pysal.contrib.gwr.gwr import FBGWR
from pysal.contrib.gwr.sel_bw import Sel_BW
import pysal as ps
import numpy as np
import geopandas as gp
import pandas as pd

In [3]:
shp = gp.read_file('/Users/toshan/dev/pysal/pysal/examples/georgia/G_utm.shp')

In [155]:
# Prep data into design matrix and coordinates

#Dependent variable
y = shp.PctBach.reshape((-1,1))

#Design matrix - covariates - intercept added automatically
pov = shp.PctPov.reshape((-1,1))
rural = shp.PctRural.reshape((-1,1))
blk = shp.PctBlack.reshape((-1,1))
X = np.hstack([pov, rural, blk])

#Coordinates for calibration points
u = shp.X
v = shp.Y
coords = zip(u,v)

coords_dict = {}
for i, x in enumerate(coords):
    coords_dict[i] = x

In [107]:
%%time 
bw = Sel_BW(coords, y, X, fb=True, constant=False)
bws = bw.search(tol_fb=1e-03, rss_score=True)
XB = bw.XB
err = bw.err 
results = FBGWR(coords, y, X, bws, XB, err, constant=False).fit()

64.0
0.104281854283
0.0342621458166
0.0130283912795
0.00957037273846
0.00674094734957
0.00332274243073
0.0111239066787
0.000960749517894
[[  43.   83.   44.]
 [  66.   98.   44.]
 [  66.  118.   44.]
 [  66.  146.   44.]
 [  66.  169.   44.]
 [  66.  169.   44.]
 [  66.  171.   47.]
 [  66.  171.   47.]]
CPU times: user 33.1 s, sys: 139 ms, total: 33.2 s
Wall time: 33 s


In [108]:
%%time 
results_Wei = FBGWR_Wei(y, X, coords_dict, tolFB=1e-03, socType=1)

0.104281828852
0.034262147915
0.0130283910987
0.00957037209451
0.0067409474738
0.00332274270222
0.0111239059726
0.000960749478577
CPU times: user 55.6 s, sys: 243 ms, total: 55.9 s
Wall time: 55.7 s


In [109]:
results.params[0]

array([ 0.57849502,  0.01962661,  0.1987632 ])

In [110]:
results_Wei.Betas[0]

array([ 0.57849505,  0.01962661,  0.19876319])

In [111]:
np.allclose(results.params, results_Wei.Betas)

True

In [112]:
bw.bw[1]

array([[  43.,   83.,   44.],
       [  66.,   98.,   44.],
       [  66.,  118.,   44.],
       [  66.,  146.,   44.],
       [  66.,  169.,   44.],
       [  66.,  169.,   44.],
       [  66.,  171.,   47.],
       [  66.,  171.,   47.]])

In [113]:
results_Wei.band_all

array([[  43.,   83.,   44.],
       [  66.,   98.,   44.],
       [  66.,  118.,   44.],
       [  66.,  146.,   44.],
       [  66.,  169.,   44.],
       [  66.,  169.,   44.],
       [  66.,  171.,   47.],
       [  66.,  171.,   47.]])

In [114]:
results.predy[0:5]

array([[  9.89976018],
       [ 10.19529001],
       [ 12.87443516],
       [ 11.82141872],
       [  8.0320105 ]])

In [115]:
results_Wei.y_pred[0:5]

array([[  9.89976054],
       [ 10.19529028],
       [ 12.87443539],
       [ 11.82141872],
       [  8.03201039]])

In [116]:
np.allclose(results.predy, results_Wei.y_pred)

True

In [117]:
bw.bw[3]

array([ 0.10428185,  0.03426215,  0.01302839,  0.00957037,  0.00674095,
        0.00332274,  0.01112391,  0.00096075])

In [118]:
results_Wei.SOC_all

array([ 0.10428183,  0.03426215,  0.01302839,  0.00957037,  0.00674095,
        0.00332274,  0.01112391,  0.00096075])

In [122]:
np.allclose(bw.bw[3], results_Wei.SOC_all)

True

In [126]:
sim = pd.read_csv('/Users/toshan/Dropbox/Articles/MC_Sims/data/variables/0_0.csv')

In [158]:
#Dependent variable
y = sim.Y.reshape((-1,1))

#Design matrix - covariates - intercept added automatically
X1 = sim.X1.reshape((-1,1))
X2 = sim.X2.reshape((-1,1))
X = np.hstack([X1, X2])

#Coordinates for calibration points
u = sim.Xcoord
v = sim.Ycoord
coords = zip(u,v)

coords_dict = {}
for i, x in enumerate(coords):
    coords_dict[i] = x

In [137]:
%%time 
bw = Sel_BW(coords, y, X, fb=True, constant=True)
bws = bw.search(tol_fb=1e-05, rss_score=True)
XB = bw.XB
err = bw.err 
results = FBGWR(coords, y, X, bws, XB, err, constant=True).fit()

146.0
0.000692626583633
0.0131164032183
0.00310245920562
0.000257446065538
0.000488325613785
4.20055728058e-05
2.75235389631e-06
[[ 171.  428.  102.]
 [ 296.  442.  102.]
 [ 359.  442.  102.]
 [ 359.  442.  102.]
 [ 375.  442.  102.]
 [ 375.  442.  102.]
 [ 375.  442.  102.]]
CPU times: user 1h 59min 25s, sys: 4min 30s, total: 2h 3min 55s
Wall time: 59min 1s


In [159]:
%%time 
X = np.concatenate([np.ones((2500)).reshape((-1,1)), X], axis=1)
results_Wei = FBGWR_Wei(y, X, coords_dict, tolFB=1e-05, socType=1, )

0.000692631587086
0.0131164052867
0.00310245935604
0.000257446162065
0.00048832561998
4.20055892427e-05
2.75235641788e-06
CPU times: user 9h 34min 26s, sys: 8min 37s, total: 9h 43min 4s
Wall time: 5h 22min 17s


In [160]:
results.params[0]

array([ 3.31194789,  0.19643385,  0.60366654])

In [161]:
results_Wei.Betas[0]

array([ 3.31194747,  0.19643384,  0.60366654])

In [162]:
np.allclose(results.params, results_Wei.Betas)

True

In [163]:
bw.bw[1]

array([[ 171.,  428.,  102.],
       [ 296.,  442.,  102.],
       [ 359.,  442.,  102.],
       [ 359.,  442.,  102.],
       [ 375.,  442.,  102.],
       [ 375.,  442.,  102.],
       [ 375.,  442.,  102.]])

In [164]:
results_Wei.band_all

array([[ 171.,  428.,  102.],
       [ 296.,  442.,  102.],
       [ 359.,  442.,  102.],
       [ 359.,  442.,  102.],
       [ 375.,  442.,  102.],
       [ 375.,  442.,  102.],
       [ 375.,  442.,  102.]])

In [173]:
results.predy[0:5]

array([[  13.58154797],
       [  18.71209421],
       [  23.86645737],
       [ 133.40999   ],
       [  71.2334284 ]])

In [174]:
results_Wei.y_pred[0:5]

array([[  13.5815474 ],
       [  18.7120939 ],
       [  23.8664568 ],
       [ 133.40999078],
       [  71.23342905]])

In [178]:
np.allclose(results.predy, results_Wei.y_pred, atol=1e-06)

True

In [169]:
bw.bw[3]

array([  6.92626584e-04,   1.31164032e-02,   3.10245921e-03,
         2.57446066e-04,   4.88325614e-04,   4.20055728e-05,
         2.75235390e-06])

In [171]:
results_Wei.SOC_all

array([  6.92631587e-04,   1.31164053e-02,   3.10245936e-03,
         2.57446162e-04,   4.88325620e-04,   4.20055892e-05,
         2.75235642e-06])

In [172]:
np.allclose(bw.bw[3], results_Wei.SOC_all)

True